In [ ]:
%matplotlib inline

from datetime import timedelta, date
import numpy as np
import xarray as xr

In [ ]:
# Using dask
import dask
from dask import compute
from dask.distributed import Client

client = Client(n_workers = 2, threads_per_worker = 2, memory_limit = "8GB")
client

In [ ]:
def percentile_ufunc(data, q, axis):
    if q == 0:
        perc = np.amin(data, axis = axis)
    elif q == 50:
        perc = np.median(data, axis = axis)
    elif q == 100:
        perc = np.amax(data, axis = axis)
    else:
        perc = np.percentile(data, q = q, axis = axis)
    return perc

def compute_percentile(data, q, dim):
    print("Computing percentile " + str(q))
    perc = xr.apply_ufunc(percentile_ufunc, data,
                          input_core_dims = [[dim]],
                          dask = 'parallelized',
                          output_dtypes = [data.dtype],
                          kwargs = {'q': q, 'axis': -1})
    return perc

In [ ]:
# Open dataset with dask
ds = xr.open_dataset("/scratch/rd/nen/perClaudia/era5/fwi_era5_1980_2016.nc", chunks = {'lat': 10, 'lon': 10})

In [ ]:
# Extract data array
fwi = ds['fwi']

In [ ]:
# Compute 90th percentile
fwi90 = compute_percentile(fwi, q = 90, dim = 'time')
x = fwi90.compute()

In [ ]:
x.to_netcdf(path = '/scratch/rd/nen/perClaudia/era5/fwi_era5_1980_2016_90th.nc')